In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from optimus import *

In [16]:
# Create optimus
op = Optimus()


             ____        __  _                     
            / __ \____  / /_(_)___ ___  __  _______
           / / / / __ \/ __/ / __ `__ \/ / / / ___/
          / /_/ / /_/ / /_/ / / / / / / /_/ (__  ) 
          \____/ .___/\__/_/_/ /_/ /_/\__,_/____/  
              /_/                                  
              
Just checking that all necessary environments vars are present...
-----
PYSPARK_PYTHON=python
SPARK_HOME=C:\opt\spark\spark-2.3.1-bin-hadoop2.7
JAVA_HOME=C:\java8
-----
Starting or getting SparkSession and SparkContext...
Setting checkpoint folder (local). If you are in a cluster change it with set_check_point_ folder(path,'hadoop').
Deleting previous folder if exists...
Creating the checkpoint directory...
Done.
Optimus successfully imported. Have fun :).


## Create dataframe
### Spark

This is ugly:

```
val someData = Seq(
  Row(8, "bat"),
  Row(64, "mouse"),
  Row(-27, "horse")
)

val someSchema = List(
  StructField("number", IntegerType, true),
  StructField("word", StringType, true)
)

val someDF = spark.createDataFrame(
  spark.sparkContext.parallelize(someData),
  StructType(someSchema)
)```

In [17]:
# Thanks Mr Powers
df = op.create.df(
    [
                ("words", "str", True),
                ("num", "int", True),
                ("animals", "str", True),
                ("thing", StringType(), True),
                ("two strings", StringType(), True),
                ("filter", StringType(), True),
                ("num 2", "string", True),
                ("date", "string", True),
                ("num 3", "string", True)
                
            ],[
                ("  I like     fish  ", 1, "dog", "&^%$#housé", "cat-car", "a","1", "20150510", "3"),
                ("    zombies", 2, "cat", "tv", "dog-tv", "b","2", "20160510", "3"),
                ("simpsons   cat lady", 2, "frog", "table","eagle-tv-plus","1","3", "20170510", "4"),
                (None, 3, "eagle", "glass", "lion-pc", "c","4", "20180510", "5"),
    
            ]
            )

df.show()

+-------------------+---+-------+----------+-------------+------+-----+--------+-----+
|              words|num|animals|     thing|  two strings|filter|num 2|    date|num 3|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+
|  I like     fish  |  1|    dog|&^%$#housé|      cat-car|     a|    1|20150510|    3|
|            zombies|  2|    cat|        tv|       dog-tv|     b|    2|20160510|    3|
|simpsons   cat lady|  2|   frog|     table|eagle-tv-plus|     1|    3|20170510|    4|
|               null|  3|  eagle|     glass|      lion-pc|     c|    4|20180510|    5|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+



In [21]:
df.dtypes

[('words', 'string'),
 ('num', 'int'),
 ('animals', 'string'),
 ('thing', 'string'),
 ('two strings', 'string'),
 ('filter', 'string'),
 ('num 2', 'string'),
 ('date', 'string'),
 ('num 3', 'string')]

In [27]:
from pyspark.sql.functions import udf

# Use udf to define a row-at-a-time udf

# Input/output are both a single double value
@udf('int')
def plus_one(v):
    return v+1

df.withColumn('new_num', plus_one(df.num)).show()

+-------------------+---+-------+----------+-------------+------+-----+--------+-----+-------+
|              words|num|animals|     thing|  two strings|filter|num 2|    date|num 3|new_num|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+-------+
|  I like     fish  |  1|    dog|&^%$#housé|      cat-car|     a|    1|20150510|    3|      2|
|            zombies|  2|    cat|        tv|       dog-tv|     b|    2|20160510|    3|      3|
|simpsons   cat lady|  2|   frog|     table|eagle-tv-plus|     1|    3|20170510|    4|      3|
|               null|  3|  eagle|     glass|      lion-pc|     c|    4|20180510|    5|      4|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+-------+



In [28]:
from pyspark.sql.functions import pandas_udf, PandasUDFType

# Use pandas_udf to define a Pandas UDF
@pandas_udf('double', PandasUDFType.SCALAR)
# Input/output are both a pandas.Series of doubles

def pandas_plus_one(v):
    return v + 1

df.withColumn('new_num', pandas_plus_one(df.num)).show()

+-------------------+---+-------+----------+-------------+------+-----+--------+-----+-------+
|              words|num|animals|     thing|  two strings|filter|num 2|    date|num 3|new_num|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+-------+
|  I like     fish  |  1|    dog|&^%$#housé|      cat-car|     a|    1|20150510|    3|    2.0|
|            zombies|  2|    cat|        tv|       dog-tv|     b|    2|20160510|    3|    3.0|
|simpsons   cat lady|  2|   frog|     table|eagle-tv-plus|     1|    3|20170510|    4|    3.0|
|               null|  3|  eagle|     glass|      lion-pc|     c|    4|20180510|    5|    4.0|
+-------------------+---+-------+----------+-------------+------+-----+--------+-----+-------+



In [7]:
!pip install pyarrow==0.9.*

  Using cached https://files.pythonhosted.org/packages/5a/e3/236653ba17aaa4b4d7a50fa262ef2aa2efc1bc470ad81120f119d9ba5c7b/pyarrow-0.9.0-cp36-cp36m-win_amd64.whl


You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [29]:
# Choose a column for analyzing
detector = op.OutlierDetector(df,"num")
# With the outliers() method you can use MAD to detect if there is an outlier in your column
detector.outliers()
# And with the run() method you can see which values are not outliers
detector.run()
# Finally with the delete_outliers() method you can delete existing outliers in your column. 
# This will modify the dataframe we have used when instantiating the OutlierDetector
# (deleting the whole row that contains the outlier value), but the original dataframe that we 
# read from disk will be intact.
detector.delete_outliers().show()

AttributeError: 'Optimus' object has no attribute 'OutlierDetector'

In [11]:
master = None
if master is None:
    master = "ASdf"
print(master)

ASdf


In [4]:
class A:
    def __init__(self, a=12):
        print(a)
        pass
    
    def turnon(self, a= 2, b=4):
        print(b)

In [5]:
a=A()
a.turnon()

12
4


In [27]:
from pyspark.sql import types as t

In [28]:
t.__all__

['DataType',
 'NullType',
 'StringType',
 'BinaryType',
 'BooleanType',
 'DateType',
 'TimestampType',
 'DecimalType',
 'DoubleType',
 'FloatType',
 'ByteType',
 'IntegerType',
 'LongType',
 'ShortType',
 'ArrayType',
 'MapType',
 'StructField',
 'StructType']

In [149]:
from pyspark.sql import functions as F

df = op.create.df(
    [
                ("num 1", "int", True), # a simple value
                ("num 2", "int", True), # multiple values
                ("num 3", "int", True), # Not values are repeated
                
            ],[

                (3,3,4),
                (3,3,3),
                (2,2,2),
                (1,2,1),
                (0,1,0)
            ]
        )


def mode(column):
    cnts = df.groupBy(column).count()
    mode = cnts.join(
        cnts.agg(F.max("count").alias("max_")), F.col("count") == F.col("max_")
    )
    # if none of the values are repeated    
    result =mode.filter(mode["count"]>1).select(column).collect()
    values = [i[0] for i in result]
    return {column:values}

df.cols().apply(["num 1"], mode)

#print(mode("num 1"))
#print (mode("num 2"))
#print (mode("num 3"))



AssertionError: col should be Column

In [140]:
print(r)

print([i[0] for i in r])


[Row(num 2=3), Row(num 2=2)]
[3, 2]
3
2


In [73]:


df.show()

+-----+-----+-----+
|num 1|num 2|num 3|
+-----+-----+-----+
|    3|    3|    4|
|    3|    3|    3|
|    2|    2|    3|
|    1|    2|    1|
+-----+-----+-----+



In [74]:
mode = cnts.join(
    cnts.agg(F.max("count").alias("max_")), F.col("count") == F.col("max_")
).select("x")

AnalysisException: "cannot resolve '`x`' given input columns: [num 1, count, max_];;\n'Project ['x]\n+- AnalysisBarrier\n      +- Join Inner, (count#736L = max_#798L)\n         :- Aggregate [num 1#654], [num 1#654, count(1) AS count#736L]\n         :  +- LogicalRDD [num 1#654, num 2#655, num 3#656], false\n         +- Aggregate [max(count#736L) AS max_#798L]\n            +- Aggregate [num 1#654], [num 1#654, count(1) AS count#736L]\n               +- LogicalRDD [num 1#654, num 2#655, num 3#656], false\n"